In [1]:
# import shutil

# # Specify the directory to delete
# dir_path = './results_bert_model_1'

# # Delete the directory and its contents
# shutil.rmtree(dir_path)

# print(f"All data in '{dir_path}' has been deleted.")

In [2]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8dlhzdch
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8dlhzdch
  Resolved https://github.com/huggingface/transformers.git to commit 94ae1ba5b55e79ba766582de8a199d8ccf24a021
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10877688 sha256=43dab52c2a53a43c818e09c4cd9e5c341179206ee2a3008343cac3f1520db208
  Stored in directory: /tmp/pip-ephem-wheel-cache-260sgxq1/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [4]:
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
#from datasets import load_dataset

# Load PAWS Subsets
#labeled_final = load_dataset("paws", "labeled_final")

#load MRPC Dataset
labeled_final = load_dataset("glue", "mrpc")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
labeled_final

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [7]:
labeled_final['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [8]:
labeled_final['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [9]:
# Define text preprocessing function
def preprocess_text(data):
    # Compile regex patterns once
    url_pattern = re.compile(r"http\S+|www\S+|https\S+")
    space_pattern = re.compile(r'\s+')
    contractions = {
        "can't": "cannot", "won't": "will not", "n't": " not", "'re": " are",
        "'s": " is", "'d": " would", "'ll": " will", "'t": " not", "'ve": " have", "'m": " am"
    }

    def clean_sentence(sentence):
        # Remove extra spaces
        sentence = space_pattern.sub(' ', sentence.strip())
        # Remove URLs
        sentence = url_pattern.sub('', sentence)
        # Convert to lowercase
        sentence = sentence.lower()
        # Normalize contractions
        sentence = ' '.join([contractions.get(word, word) for word in sentence.split()])
        return sentence

    # Apply preprocessing to both sentence1 and sentence2
    data['sentence1'] = [clean_sentence(sent) for sent in data['sentence1']]
    data['sentence2'] = [clean_sentence(sent) for sent in data['sentence2']]
    return data

In [10]:
# Apply preprocessing to the dataset
train_dataset = labeled_final["train"].map(preprocess_text, batched=True)
valid_dataset = labeled_final["validation"].map(preprocess_text, batched=True)
test_dataset = labeled_final["test"].map(preprocess_text, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [11]:
train_dataset[0]

{'sentence1': 'amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'referring to him as only " the witness " , amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [12]:
# train = labeled_final['train'].select(range(500))
# test = labeled_final['test'].select(range(50))
# valid = labeled_final['validation'].select(range(50))
#---

# train = train_dataset.select(range(1000))
# valid = valid_dataset.select(range(50))
# test = test_dataset.select(range(50))
#---
train = train_dataset
valid = valid_dataset
test = test_dataset

In [13]:
train[0]

{'sentence1': 'amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'referring to him as only " the witness " , amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [14]:
from google.colab import userdata
HF_TOKEN= userdata.get('HuggingFace')

from huggingface_hub import login
login(token=HF_TOKEN)

In [15]:
# Load tokenizer
model_name = "google-t5/t5-base"   #"answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding="max_length", max_length=128)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [16]:
tokenized_train = train.map(tokenize_function, batched=True)
tokenized_valid = valid.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [17]:
tokenized_train

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 3668
})

In [18]:
# Remove unnecessary columns and rename label column
tokenized_train = tokenized_train.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch")
tokenized_train




Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3668
})

In [19]:
tokenized_valid = tokenized_valid.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_valid = tokenized_valid.rename_column("label", "labels")
tokenized_valid.set_format("torch")

#----------
tokenized_test = tokenized_test.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_test = tokenized_test.rename_column("label", "labels")
tokenized_test.set_format("torch")
tokenized_test

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1725
})

In [20]:
tokenized_valid[0]

{'labels': tensor(1),
 'input_ids': tensor([   3,   88,  243,    8,  542, 5114, 6253,  268,  744,   59, 1400,    8,
          349,   19,  307,   18, 1987, 1170, 1998,    3,    5,    1,   96,    8,
          542, 5114, 6253,  268,  405,   59, 1400,   69,  307,   18, 1987, 1170,
         1998,    3,    5,    1,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [21]:
# Data collator for dynamic padding
#from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [48]:
#from transformers import TrainingArguments
# Define training arguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_T5_model",         # Output directory for saved models
    learning_rate=1e-6,          # Learning rate
    per_device_train_batch_size=8, # Training batch size
    per_device_eval_batch_size=8, # Evaluation batch size
    num_train_epochs=20,            # Number of training epochs
    weight_decay=0.01,             # Weight decay
    #save_total_limit=2,            # Save only the 2 most recent models
    logging_dir="./logs",          # Log directory
    logging_steps=50,              # Log every 50 steps
    eval_strategy="epoch",   # Evaluate after each epoch
    save_strategy="epoch",         # Change save_strategy to 'epoch' to match eval_strategy
    load_best_model_at_end=True,    # Load the best model at the end of training
    metric_for_best_model="eval_loss",
    fp16=True,                       # Enabled mixed precision training
    gradient_accumulation_steps=2, # Added gradient accumulation
    #gradient_checkpointing=True,
    max_grad_norm=1.0,  # Clip gradients with a maximum norm of 1.0
    report_to="none",
)

In [49]:
# Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Explicitly set for binary classification
)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"


In [44]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define a metric function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions[0].argmax(-1)  # Get the class with the highest score

    # Calculate accuracy, precision, recall, and F1
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [51]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid, # Assuming you have a validation set
    #tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Early stopping
)

In [46]:
#!nvidia-smi

In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.631700,0.623260,0.683824,0.683824,1.000000,0.812227
2,0.632100,0.619489,0.683824,0.683824,1.000000,0.812227
3,0.634800,0.615157,0.683824,0.683824,1.000000,0.812227
4,0.633100,0.611374,0.683824,0.683824,1.000000,0.812227
5,0.618800,0.607167,0.683824,0.683824,1.000000,0.812227
6,0.606900,0.604141,0.688725,0.687192,1.000000,0.814599
7,0.613600,0.600804,0.696078,0.692308,1.000000,0.818182
8,0.617700,0.596936,0.696078,0.692308,1.000000,0.818182
9,0.617800,0.593883,0.696078,0.693267,0.996416,0.817647
10,0.611600,0.591154,0.693627,0.692500,0.992832,0.815906


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=4140, training_loss=0.6092731144117273, metrics={'train_runtime': 2379.3887, 'train_samples_per_second': 30.831, 'train_steps_per_second': 1.925, 'total_flos': 1.008148374116352e+16, 'train_loss': 0.6092731144117273, 'epoch': 18.0})

In [34]:
valid_results = trainer.evaluate(tokenized_valid)
print("Evaluation Results:", valid_results)

Evaluation Results: {'eval_loss': 0.37970849871635437, 'eval_accuracy': 0.8627450980392157, 'eval_precision': 0.8631921824104235, 'eval_recall': 0.9498207885304659, 'eval_f1': 0.9044368600682594, 'eval_runtime': 3.2596, 'eval_samples_per_second': 125.167, 'eval_steps_per_second': 15.646, 'epoch': 9.0}


In [35]:
# test_results = trainer.predict(tokenized_test_datasets)
# metrics = compute_metrics(test_results)
# print(metrics)

test_results = trainer.evaluate(tokenized_test)
print("Test Results:", test_results)

Test Results: {'eval_loss': 0.4353073239326477, 'eval_accuracy': 0.8388405797101449, 'eval_precision': 0.8473221422861711, 'eval_recall': 0.9241499564080209, 'eval_f1': 0.8840700583819849, 'eval_runtime': 14.2814, 'eval_samples_per_second': 120.787, 'eval_steps_per_second': 15.125, 'epoch': 9.0}


In [ ]:
# # Generate predictions for the test set
# predictions = test_results.predictions.argmax(-1)  # Predicted labels
# true_labels = test_results.label_ids               # True labels

# # Pair predictions with true labels
# for i in range(len(predictions)):
#     print(f"Sentence1: {tokenized_test_datasets[i]['sentence1']}")
#     print(f"Sentence2: {tokenized_test_datasets[i]['sentence2']}")
#     print(f"True Label: {true_labels[i]}, Predicted Label: {predictions[i]}\n")